In [1]:
%load_ext autoreload
%autoreload 2
from TNR import *

In [2]:
# htemp=KP(s0,sx,sx,s0)+KP(s0,sy,sy,s0)+KP(s0,sz,sz,s0)
# htemp=KP(sx,sx,s0,s0)+KP(sy,sy,s0,s0)+KP(sz,sz,s0,s0)
# htemp=KP(sz,sz)+(KP(sx,s0)+KP(s0,sx))/4

# d=int(np.sqrt(htemp.shape[0]))
# Ainit=expm(-htemp).reshape(d,d,d,d)
# A0=Asplit(Ainit,chi)

na=np.newaxis
s=np.array([1,-1])
Tc=2/np.log(1+np.sqrt(2))
T=1*Tc
A0=np.exp(-1/T*(s[:,na,na,na]*s[na,:,na,na]+s[na,:,na,na]*s[na,na,:,na]\
                +s[na,na,:,na]*s[na,na,na,:]+s[na,na,na,:]*s[:,na,na,na]))
A0/=np.linalg.norm(A0)

chi={}
chi['w'],chi['y'],chi['u'],chi['v']=6,6,6,6
chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']

In [3]:
RGsteps=6
A_list=[None]*(RGsteps+1)
Anorm_list=[None]*(RGsteps+1)
B_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list=[[None]*RGsteps for i in range(10)]
A_list[0]=A0

for i in range(RGsteps):
    print('RG step: ',i)
    Aold=A_list[i]
    B,U,vl,vr,yl,yr,D,w,u,v,A,Anorm=doTNR(Aold,chi)
    B_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i],A_list[i+1],Anorm_list[i+1]=\
    B,U,vl,vr,yl,yr,D,w,u,v,A,Anorm
    print()

RG step:  0
iter=0, U.Uenv=0.58
iter=100, U.Uenv=0.58
iter=200, U.Uenv=0.58
iter=300, U.Uenv=0.58
error in U, vl, vr optimization: 4.09413e-12
error in yl, yr, D optimization: 0.00259533
error in w optimization: 0.0453499
Anew.shape!=Aold.shape, skip fix_gauge

RG step:  1
iter=0, U.Uenv=0.488667
iter=100, U.Uenv=0.489098
iter=200, U.Uenv=0.489099
iter=300, U.Uenv=0.4891
iter=400, U.Uenv=0.4891
iter=500, U.Uenv=0.4891
error in U, vl, vr optimization: 0.00548505
error in yl, yr, D optimization: 0.013349
error in w optimization: 0.0222184
iter=0, u.uenv=0.337954, v.venv=0.95806
iter=1000, u.uenv=0.969214, v.venv=0.969214
iter=2000, u.uenv=0.969214, v.venv=0.969214
Adiff 0.248138

RG step:  2
iter=0, U.Uenv=0.527759
iter=100, U.Uenv=0.527833
iter=200, U.Uenv=0.527833
iter=300, U.Uenv=0.527833
error in U, vl, vr optimization: 0.00250269
error in yl, yr, D optimization: 0.00273869
error in w optimization: 0.0188834
iter=0, u.uenv=0.968266, v.venv=0.971475
iter=1000, u.uenv=0.970696, v.venv=

In [4]:
# def Az_(Ainit,chi):
#     #input: Ainit, chi, op: operator acting on a leg
#     #output:up-down symmetric Aout
#     chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']
#     As=Ainit.shape
    
#     #vertical cut
#     Av=Ainit.reshape(As[0]*As[1],As[2]*As[3])
#     dtemp,vtemp=eigCut(Av,chiw)
#     v=(vtemp*np.sqrt(dtemp)).reshape(As[0],As[1],min(chiw,As[0]*As[1]))

#     #horizontal cut
#     vl2,w,vr2= h_cut(Ainit,chiy)
#     Aout=np.einsum(vr2,[4,5,0],v,[5,7,1],vl2*w,[6,7,2],v.conj(),[4,6,3])
#     Azout=0.5*(np.einsum(vr2,[4,5,0],v,[5,7,1],sz@vl2*w,[6,7,2],v.conj(),[4,6,3])+np.einsum(vr2,[4,5,0],v,[5,7,1],vl2*w,[6,7,2],sz@v.conj(),[4,6,3]))
#     Anorm=np.linalg.norm(Aout)
#     return Azout/Anorm

# A0=Asplit(Ainit,chi)
# Az=Az_(Ainit,chi)
# def MZ_(A0,Az):
#     return np.einsum(Az,[2,9,8,0],A0,[8,10,3,1],A0,[4,6,11,9],A0,[11,7,5,10])

In [17]:
#eval mag
tensor_list={}
tensor_list['A'],tensor_list['U'],tensor_list['vl'],tensor_list['vr'],tensor_list['yl'],tensor_list['yr'],tensor_list['D'],tensor_list['w'],tensor_list['u'],tensor_list['v']=A_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list
op=sz
eval_op(op,tensor_list)

RGsteps: 0 , <O> = (1.4212960490389293e-05+0j)
RGsteps: 1 , <O> = (0.0009028984210861405+0j)
RGsteps: 2 , <O> = (0.004384106686966572+0j)
RGsteps: 3 , <O> = (0.01638443445884618+0j)
RGsteps: 4 , <O> = (0.05657046213368866+0j)
RGsteps: 5 , <O> = (0.18388684929804064+0j)


(0.18388684929804064-1.103265434319907e-19j)

In [7]:
#exact value
mag_exact=(1-np.sinh(2/T)**(-4))**(1/8)
print(mag_exact)

nan


c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


scaling dimensions

In [13]:
i=3
A=A_list[i+1]
Aold,U,vl,vr,yl,yr,D,w,u,v=A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i]

gl=gl_(vl)
gr=gr_(vr)
gu=gu_(Aold,U,vl,vr)
gnw=gnw_(vl,vr,yr,D,w)
gne=gne_(vl,vr,yl,D,w)
gnw,gne=gauge_(gnw,gne,u,v)

def scaleop(v):
    M=v.reshape(chiw,chiw,chiy,chiy,chiy,chiy,chiw,chiw)
    Mscaled=rg_(M,gl,gr,gu,gnw,gne)
    return Mscaled.flatten()

n=chiw**4*chiy**4

In [15]:
import scipy
from scipy.sparse.linalg import LinearOperator

N_level=100
Atemp = LinearOperator((n,n), matvec = scaleop, dtype='complex')
ew = scipy.sparse.linalg.eigs(Atemp, k=N_level, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))

array([-0.        ,  0.12896736,  1.00033299,  1.12116407,  1.12141163,
        1.99889291,  1.99930069,  2.00120964,  2.00121109,  2.11838727,
        2.12264671,  2.12951043,  2.99357471,  2.99388519,  3.00051744,
        3.00216003,  3.00295551,  3.09813267,  3.12658064,  3.12660881,
        3.130263  ,  3.13157471,  3.13413701,  3.98227535,  3.98340818,
        3.9922838 ,  3.99400632,  3.9942293 ,  3.99445219,  4.00098392,
        4.0014756 ,  4.00535889,  4.05659084,  4.1064788 ,  4.10765815,
        4.11969705,  4.12397837,  4.12943898,  4.13575797,  4.14180143,
        4.14249737,  4.89492834,  4.9342542 ,  4.93751714,  4.94794961,
        4.95031252,  4.98272165,  4.9843899 ,  4.99182022,  4.99480473,
        4.99551623,  4.99568795,  5.00191243,  5.00273679,  5.00633475,
        5.06151567,  5.06583195,  5.09953454,  5.10700774,  5.10763618,
        5.12252753,  5.12292191,  5.13679564,  5.13825183,  5.14416891,
        5.14639191,  5.14771142,  5.18492056,  5.65456615,  5.80

In [11]:
# compress legs

# def wrenv_(A):
#     return np.einsum(A,[5,6,0,8],A,[4,7,1,6],A.conj(),[4,7,3,9],A.conj(),[5,9,2,8])
# def wlenv_(A):
#     return np.einsum(A,[0,6,5,8],A,[1,7,4,6],A.conj(),[3,7,4,9],A.conj(),[2,9,5,8])
# def wvenv_(A):
#     return np.einsum(A,[8,0,6,5],A,[6,1,7,4],A.conj(),[8,2,9,5],A.conj(),[9,3,7,4])
# chik=20
# wrenv=wrenv_(A)
# _,wr=compress(wrenv,chik)
# wlenv=wlenv_(A)
# _,wl=compress(wlenv,chik)
# wvenv=wvenv_(A)
# _,wv=compress(wvenv,chik)

# wrenv=wrenv_(Aold)
# _,wrold=compress(wrenv,chik)
# wlenv=wlenv_(Aold)
# _,wlold=compress(wlenv,chik)
# wvenv=wvenv_(Aold)
# _,wvold=compress(wvenv,chik)

# glw=np.einsum(gl,[3,4,1,2],wrold.conj(),[3,4,0])
# grw=np.einsum(gr,[3,4,1,2],wlold.conj(),[3,4,0])
# guw=np.einsum(gu,[3,4,1,2],wvold.conj(),[3,4,0])

# def scaleop(v):
#     v4=v.reshape(chik,chik,chik,chik)
#     out=np.einsum(v4,[8,9,10,11],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wl,[4,5,0],wr,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3],optimize=('greedy',2**100))
#     return out.flatten()
# def rscaleop(v):
#     v4=v.reshape(chik,chik,chik,chik)
#     out=np.einsum(v4.conj(),[0,1,2,3],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wl,[4,5,0],wr,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3],optimize=('greedy',2**100))
#     return out.flatten().conj()

# n=chik**4